# Text Rank 的处理流程（single-domain-multiple-documents）

![](https://cdn.analyticsvidhya.com/wp-content/uploads/2018/10/block_3.png)


这是一个 single-domain-multiple-documents 的摘要任务。我们会使用多个文章作为输入，并生成一个要点式的摘要。

- 1.把所有的文章中的句子，连接起来
- 2.把文本分割成一个个的句子
- 3.对每个句子做向量化
- 4.计算句子之间的相似度，并存储到矩阵中
- 5.把相似度矩阵转化为图，其中句子是节点，相似度是节点间的权重
- 6.取到 top-k 权重的句子，作为最终的摘要

## 读取并探索数据集

In [1]:
!pip3 install pandas -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
import pandas as pd

df = pd.read_csv('nba-articles.csv')

In [3]:
df.head()

,article_id,article_text
0,1,浓眉哥”安东尼-戴维斯自从出道以来，就倍受全美球迷追捧。美国球评称他为美国内线最后的希望。浓...
1,2,北京时间11月18日，洛杉矶湖人以102-109不敌密尔沃基雄鹿。赛后，湖人主帅沃格尔谈到了...
2,3,北京时间11月18日，密尔沃基雄鹿以109-102战胜了洛杉矶湖人，字母哥在今天的比赛中23...
3,4,北京时间11月18日，密尔沃基雄鹿以109-102战胜了洛杉矶湖人。此役，字母哥23投18中...
4,5,北京时间11月18日，湖人败给雄鹿，拉塞尔-威斯布鲁克表现不是特别抢眼，但失误赛季第二少，助...


## 文本转化为句子

In [4]:
## 标准化文本
def as_text(v):  ## 生成unicode字符串
    if v is None:
        return None
    elif isinstance(v, bytes):
        return v.decode('utf-8', errors='ignore')
    elif isinstance(v, str):
        return v
    else:
        raise ValueError('Unknown type %r' % type(v))

        
text_type    = str
string_types = (str,)
xrange       = range


def is_text(v):
    return isinstance(v, text_type)

In [5]:
class SentenceSegmentation(object):
    """ 分句 """
    
    def __init__(self, delimiters=['?', '!', ';', '？', '！', '。', '；', '……', '…', '\n']
):
        """
        Keyword arguments:
        delimiters -- 可迭代对象，用来拆分句子
        """
        self.delimiters = set([as_text(item) for item in delimiters])
    
    def segment(self, text):
        res = [as_text(text)]

        for sep in self.delimiters:
            text, res = res, []
            for seq in text:
                res += seq.split(sep)
        res = [s.strip() for s in res if len(s.strip()) > 0]
        return res 

In [6]:
ss = SentenceSegmentation()
ss.segment(df['article_text'][1])

['北京时间11月18日，洛杉矶湖人以102-109不敌密尔沃基雄鹿',
 '赛后，湖人主帅沃格尔谈到了霍顿-塔克',
 '沃格尔说：“塔克在过去两场比赛中都表现得很出色，即使勒布朗回来，他也有充分的理由首发出战”',
 '“在替补席上，塔克也有得分和组织的任务，但是现在的塔克足够优秀，他也可以获得首发的位置”',
 '此役，塔克出战38分58秒，18投9中得到了25分12篮板3助攻2抢断',
 '在上一场比赛中，塔克得到了职业生涯最高的28分',
 '值得一提的是，在下一场比赛中，詹姆斯还未确定是否出战']

In [7]:
# 把所有文章的句子聚合起来
sentences = []


for s in df['article_text']:
    sentences.append(ss.segment(s))  
    
sentences

[['浓眉哥”安东尼-戴维斯自从出道以来，就倍受全美球迷追捧',
  '美国球评称他为美国内线最后的希望',
  '浓眉也确实争气，第二个赛季就打出了场均20.8分10篮板2.8封盖的华丽数据',
  '球迷们把浓眉选进了全明星阵容',
  '美国媒体也毫不客气，看浓眉就像看自家孩子一样',
  '2014-2015赛季，在浓眉入行的第三年，ESPN就把浓眉排在了自己百大球员的第三名',
  '在他身后的有杜兰特/哈登/库里/威少，一干成名已久且当时或日后都拿到了MVP的超级巨星',
  'ESPN当时的解释是，在浓眉当时的年纪，只有勒布朗-詹姆斯和魔术师的胜利贡献值比浓眉更高',
  '从那以后，浓眉哥在ESPN的排名就没低于过前六（期间两次被排在第二，2015年和2020年）',
  '但是就在本赛季开始前，ESPN把浓眉排在了第九，这几乎是浓眉成名后，被媒体最看衰的一次',
  '新科MVP约基奇，MVP排行榜第二的恩比德，率队夺冠的字母哥，都没什么争议的排在了字母哥之前',
  '浓眉哥还是全美内线最后的希望吗',
  '考虑道其他几大内线的国籍，这么说或许没问题，美国也真没拿得出手的内线了，不然梦之队也不会召唤麦基这水平的球员了',
  '但是如果还说浓眉是全NBA最好的内线',
  '看完了今天雄鹿和湖人的比赛，即使是湖人球迷也沉默不语了',
  '现在的浓眉哥，距离最强内线的称号真的是越来越远了',
  '这个赛季，湖人队交易来了威少，将19-20赛季的夺冠阵容几乎全部拆散',
  '在引援方面湖人引进的几乎全是后卫，身高体重合格的锋线只有阿里扎一个人，1米93的贝兹莫尔都得顶到前锋位置了',
  '年轻时候防守都不及格的安东尼都被迫在最近逼成了球队的首发大前锋',
  '这一套头重脚轻的阵容，素以防守调教起家的沃格尔也玩不转了',
  '湖人队如今每场比赛让对方在篮下的出手是全联盟最多的，原因很简单：球队的外线防守几乎被人一步就过，四处漏风，对方的持球型锋线谁都可以轻松突破第一道锋线杀入禁区',
  '湖人队本赛季每场比赛丢112.3分已经排到了联盟第三，球队的防守效率也只是联盟第15',
  '19-20赛季，湖人队的防守效率联盟第三',
  '上个赛季，湖人防守效率联盟第一',
  '本赛季，哪怕有戴维斯这么个最佳防守球员级别的内线大闸在，也防不

In [8]:
def flatten(items):
    for x in items:
        # 终止条件，检验是否为可迭代对象
        if hasattr(x,'__iter__') and not isinstance(x, (str, bytes)):
            yield from flatten(x)
        else:
            yield x

In [9]:
# 拉平列表
sentences = list(flatten(sentences))

In [10]:
sentences[:5]

['浓眉哥”安东尼-戴维斯自从出道以来，就倍受全美球迷追捧',
 '美国球评称他为美国内线最后的希望',
 '浓眉也确实争气，第二个赛季就打出了场均20.8分10篮板2.8封盖的华丽数据',
 '球迷们把浓眉选进了全明星阵容',
 '美国媒体也毫不客气，看浓眉就像看自家孩子一样']

In [11]:
len(sentences)

116

## 对所有句子进行分词

In [12]:
!pip3 install jieba -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [13]:
import jieba.posseg as pseg
import codecs
import os


def get_default_stop_words_file():
    d = os.path.dirname(os.path.realpath('text-rank'))
    return os.path.join(d, 'stopwords.txt')

class WordSegmentation(object):
    """ 分词 """
    
    def __init__(self, stop_words_file = None, allow_speech_tags = ['an', 'i', 'j', 'l', 'n', 'nr', 'nrfg', 'ns', 'nt', 'nz', 't', 'v', 'vd', 'vn', 'eng']):
        """
        Keyword arguments:
        stop_words_file    -- 保存停止词的文件路径，utf8编码，每行一个停止词。若不是str类型，则使用默认的停止词
        allow_speech_tags  -- 词性列表，用于过滤
        """     
        
        allow_speech_tags = [as_text(item) for item in allow_speech_tags]

        self.default_speech_tag_filter = allow_speech_tags
        self.stop_words = set()
        self.stop_words_file = get_default_stop_words_file()
        if type(stop_words_file) is str:
            self.stop_words_file = stop_words_file
        for word in codecs.open(self.stop_words_file, 'r', 'utf-8', 'ignore'):
            self.stop_words.add(word.strip())
    
    def segment(self, text, lower = True, use_stop_words = True, use_speech_tags_filter = False):
        """对一段文本进行分词，返回list类型的分词结果

        Keyword arguments:
        lower                  -- 是否将单词小写（针对英文）
        use_stop_words         -- 若为True，则利用停止词集合来过滤（去掉停止词）
        use_speech_tags_filter -- 是否基于词性进行过滤。若为True，则使用self.default_speech_tag_filter过滤。否则，不过滤。    
        """
        text = as_text(text)
        jieba_result = pseg.cut(text)
        
        if use_speech_tags_filter == True:
            jieba_result = [w for w in jieba_result if w.flag in self.default_speech_tag_filter]
        else:
            jieba_result = [w for w in jieba_result]

        # 去除特殊符号
        word_list = [w.word.strip() for w in jieba_result if w.flag!='x']
        word_list = [word for word in word_list if len(word)>0]
        
        if lower:
            word_list = [word.lower() for word in word_list]

        if use_stop_words:
            word_list = [word.strip() for word in word_list if word.strip() not in self.stop_words]

        return word_list
        
    def segment_sentences(self, sentences, lower=True, use_stop_words=True, use_speech_tags_filter=False):
        """将列表sequences中的每个元素/句子转换为由单词构成的列表。
        
        sequences -- 列表，每个元素是一个句子（字符串类型）
        """
        
        res = []
        for sentence in sentences:
            res.append(self.segment(text=sentence, 
                                    lower=lower, 
                                    use_stop_words=use_stop_words, 
                                    use_speech_tags_filter=use_speech_tags_filter))
        return res

In [14]:
ws = WordSegmentation()
words = ws.segment_sentences(sentences)
print(words)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.384 seconds.
Prefix dict has been built successfully.


[['浓眉', '哥', '安东尼', '戴维斯', '出', '道', '以来', '倍受', '全美', '球迷', '追捧'], ['美国', '球', '评称', '美国', '内线', '最后', '希望'], ['浓眉', '确实', '争气', '第二个', '赛季', '打出', '了场', '均', '20.8', '分', '10', '篮板', '2.8', '封盖', '华丽', '数据'], ['球迷', '浓眉', '选进', '全', '明星阵容'], ['美国', '媒体', '毫不客气', '看', '浓眉', '看', '孩子'], ['2014', '2015', '赛季', '浓眉', '入行', '第三年', 'espn', '浓眉', '排在', '百大', '球员', '第三名'], ['身后', '杜兰特', '哈登', '库里', '威', '少', '干', '成名', '已', '久', '当时', '日后', '都', '拿到', 'mvp', '超级', '巨星'], ['espn', '当时', '解释', '浓眉', '当时', '年纪', '勒布朗', '詹姆斯', '魔术师', '胜利', '贡献', '值比', '浓眉', '更', '高'], ['从那以后', '浓眉', '哥', 'espn', '排名', '没', '低于', '过前', '六', '期间', '两次', '排在', '第二', '2015', '年', '2020', '年'], ['本赛季', '前', 'espn', '浓眉', '排在', '第九', '几乎', '浓眉', '成名', '后', '媒体', '最', '看衰', '一次'], ['新科', 'mvp', '约基', '奇', 'mvp', '排行榜', '第二', '恩', '比德', '率队', '夺冠', '字母', '哥', '都', '没什么', '争议', '排在', '字母', '哥', '之前'], ['浓眉', '哥', '全美', '内线', '最后', '希望'], ['考虑', '道', '大', '内线', '国籍', '说', '或许', '没', '问题', '美国', '真', '没', '出手', '内线', '梦之队'

## 下载 Glove 中文词向量

[中文词向量下载地址](https://github.com/Embedding/Chinese-Word-Vectors)

In [15]:
# !wget http://aimaksen.bslience.cn/sgns.wiki.word.bz2

In [16]:
! bunzip2 sgns.wiki.word.bz2

bunzip2: Can't open input file sgns.wiki.word.bz2: No such file or directory.


In [17]:
import numpy as np


# Extract word vectors
word_embeddings = {}
f = open('sgns.wiki.word', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [18]:
len(word_embeddings)

352163

## 句子的向量表示

In [19]:
sentence_vectors = []

for i in words:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((300,))) for w in i])/(len(i)+0.001)
    else:
        v = np.zeros((300,))

    sentence_vectors.append(v)

In [20]:
len(sentence_vectors)

116

In [21]:
sentence_vectors[:2]

[array([ 0.13509555,  0.16762212, -0.13855885,  0.04146333,  0.10073085,
         0.00951659, -0.2506782 , -0.10022862, -0.02341496, -0.13381875,
        -0.07906353,  0.00769676,  0.00259358,  0.28280473, -0.09426296,
         0.07290537, -0.10634705, -0.23625089,  0.12549394,  0.15635297,
         0.03257395,  0.17713143,  0.05828978,  0.10605808,  0.01918762,
        -0.1489031 ,  0.04032315, -0.19743878,  0.05144577, -0.1445034 ,
         0.11960395, -0.12295799, -0.07967439,  0.08989102,  0.17582846,
         0.11723988,  0.22522315,  0.0086774 , -0.28286803,  0.06486292,
        -0.08689301, -0.15750304, -0.06989282,  0.00045659,  0.19347657,
         0.07916716,  0.0708072 ,  0.15490519, -0.01492464, -0.27415547,
        -0.13504653,  0.10740814, -0.0360749 ,  0.07881547, -0.00441814,
         0.09027706, -0.07603191,  0.003977  , -0.2035883 ,  0.360943  ,
        -0.05830797, -0.14227706, -0.09754686, -0.20729113, -0.1192568 ,
        -0.09305036,  0.05117926, -0.06525925,  0.0

## 计算相似度矩阵

计算句子之间的相似度。

In [22]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [23]:
sim_mat.shape

(116, 116)

In [24]:
!pip3 install scikit-learn -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [25]:
from sklearn.metrics.pairwise import cosine_similarity

In [26]:
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,300), sentence_vectors[j].reshape(1,300))[0,0]

In [27]:
sim_mat.shape

(116, 116)

## 使用 PageRank 算法

In [28]:
!pip3 install networkx -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [29]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)
scores

{0: 0.008374943442106517,
 1: 0.007978205307968224,
 2: 0.009207523379251372,
 3: 0.006999123048343878,
 4: 0.007149307086544447,
 5: 0.008328962440949581,
 6: 0.008977194683099255,
 7: 0.008478785557742802,
 8: 0.008604739608661444,
 9: 0.008800946945173844,
 10: 0.008714062958314782,
 11: 0.008096660284159158,
 12: 0.00904684969219654,
 13: 0.008186422791110759,
 14: 0.00853655873416692,
 15: 0.008113322342993251,
 16: 0.009161725924405407,
 17: 0.009430311274159294,
 18: 0.009066001076271061,
 19: 0.008326791725156046,
 20: 0.00964562678926463,
 21: 0.009474077745634005,
 22: 0.008858956906685742,
 23: 0.008791475465876858,
 24: 0.009160722116273376,
 25: 0.009604468025192433,
 26: 0.008851034361431084,
 27: 0.007950903823582283,
 28: 0.009443924083227162,
 29: 0.008765515542341049,
 30: 0.006984972531393602,
 31: 0.008557127675575328,
 32: 0.00831015269752609,
 33: 0.007919780606325973,
 34: 0.008297521189016216,
 35: 0.009489972168748223,
 36: 0.009439877133544246,
 37: 0.00917188

## 文本抽取

In [30]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [31]:
# Specify number of sentences to form the summary
sn = 10

# Generate summary
for i in range(sn):
    print(ranked_sentences[i][1])

北京时间11月18日，亚特兰大老鹰队在主场以110比99取得胜利，本场比赛老鹰队球星特雷-杨拿到了18分外加11次助攻，成为球队取胜的关键球员
北京时间11月18日，湖人败给雄鹿，拉塞尔-威斯布鲁克表现不是特别抢眼，但失误赛季第二少，助攻赛季最高
因此，字母哥成为了马布里在2005年对阵湖人的比赛之后，首位能够对阵湖人的时候砍下45+分且命中率75+%的球员
湖人队如今每场比赛让对方在篮下的出手是全联盟最多的，原因很简单：球队的外线防守几乎被人一步就过，四处漏风，对方的持球型锋线谁都可以轻松突破第一道锋线杀入禁区
在防守端浓眉已经算拼尽全力了，可是效果却惨不忍睹：本赛季浓眉在场的时候，湖人每百回合丢109.5分
因此，相比起字母哥的出色状态，浓眉哥在本赛季以来的状态一般，而本场比赛浓眉哥则是15投9中，贡献了18分9板4助2帽的数据
可本赛季，他前27次三分出手只命中了4球，创下了NBA历史赛季前25次出手三分以上的命中率新低
湖人队本赛季每场比赛丢112.3分已经排到了联盟第三，球队的防守效率也只是联盟第15
今天字母哥上半场几乎球球都冲着浓眉去，浓眉却完全做不出有效干扰，让字母哥半场13投12中，彻底打爆了湖人禁区
今天对阵雄鹿赛前，媒体晒出数据：字母哥最近三场命中的三分数量（6记）等于浓眉整个赛季命中的三分数量（6记）


# Text Rank 的处理流程（single-domain-single-document）

- 1.拿出一篇文章
- 2.把文本分割成一个个的句子并分词
- 3.对每个单词和句子做向量化
- 4.计算单词/句子之间的相似度，并存储到矩阵中
- 5.把相似度矩阵转化为图，其中单词/句子是节点，相似度是节点间的权重
- 6.取到 top-k 权重的单词/句子，作为最终的关键词和关键句

## 获取第一篇文章

In [32]:
df = pd.read_csv('nba-articles.csv')
article1 = df['article_text'][2]
article1

'北京时间11月18日，密尔沃基雄鹿以109-102战胜了洛杉矶湖人，字母哥在今天的比赛中23投18中，三分球 4投3中，砍下47分9篮板。因此，美媒统计了关于字母哥的一项数据，他也成为了马布里之后能够砍下这项数据的第一人。\r\n\u3000\u3000根据数据统计，在今天的比赛中，字母哥轰下了45分且命中率达到75%以上。因此，字母哥成为了马布里在2005年对阵湖人的比赛之后，首位能够对阵湖人的时候砍下45+分且命中率75+%的球员。\r\n\u3000\u3000最终，在字母哥的带领下，雄鹿有惊无险在主场战胜了湖人。'

## 对所有句子进行分词

In [33]:
ss = SentenceSegmentation()
sentences = ss.segment(article1)
ws = WordSegmentation()
words = ws.segment_sentences(sentences)
words[0:5]

[['北京',
  '时间',
  '11',
  '月',
  '18',
  '日',
  '密尔沃基',
  '雄鹿',
  '109',
  '102',
  '战胜',
  '洛杉矶',
  '湖人',
  '字母',
  '哥',
  '今天',
  '比赛',
  '中',
  '23',
  '投',
  '18',
  '中',
  '三分球',
  '4',
  '投',
  '3',
  '中',
  '砍',
  '下',
  '47',
  '分',
  '9',
  '篮板'],
 ['美',
  '媒',
  '统计',
  '字母',
  '哥',
  '一项',
  '数据',
  '成为',
  '马布里',
  '之后',
  '能够',
  '砍',
  '下',
  '这项',
  '数据',
  '第一',
  '人'],
 ['数据',
  '统计',
  '今天',
  '比赛',
  '中',
  '字母',
  '哥',
  '轰下',
  '45',
  '分且',
  '命中率',
  '达到',
  '75',
  '以上'],
 ['字母',
  '哥',
  '成为',
  '马布里',
  '2005',
  '年',
  '对阵',
  '湖人',
  '比赛',
  '之后',
  '首位',
  '能够',
  '对阵',
  '湖人',
  '砍',
  '下',
  '45',
  '分且',
  '命中率',
  '75',
  '球员'],
 ['最终', '字母', '哥', '带领', '下', '雄鹿', '有惊无险', '主场', '战胜', '湖人']]

In [34]:
def flatten(items):
    for x in items:
        # 终止条件，检验是否为可迭代对象
        if hasattr(x,'__iter__') and not isinstance(x, (str, bytes)):
            yield from flatten(x)
        else:
            yield x

In [35]:
flatten_words = list(set(flatten(words)))

In [36]:
flatten_words[:5]

['102', '三分球', '洛杉矶', '篮板', '能够']

## 使用向量表示单词

In [37]:
word_vectors = []

for i in flatten_words:
    v = sum([word_embeddings.get(w, np.zeros((300,))) for w in i])/(len(i)+0.001)
    word_vectors.append(v)

In [38]:
len(word_vectors)

57

## 计算相似度矩阵

计算单词之间的相似度。

In [39]:
# similarity matrix
sim_mat = np.zeros([len(flatten_words), len(flatten_words)])
sim_mat.shape

(57, 57)

In [40]:
for i in range(len(flatten_words)):
    for j in range(len(flatten_words)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(word_vectors[i].reshape(1,300), word_vectors[j].reshape(1,300))[0,0]

In [41]:
sim_mat.shape

(57, 57)

## 使用 PageRank 算法

In [42]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

## 关键词抽取

In [45]:
ranked_words = sorted(((scores[i],s) for i,s in enumerate(flatten_words)), reverse=True)

# Specify number of sentences to form the summary
sn = 10

# Generate summary
for i in range(sn):
    print(ranked_words[i][1])

109
45
18
102
47
2005
75
4
23
三分球


## 抽取关键句

In [ ]:
words[0:5]

In [ ]:
sentence_vectors = []

for i in words:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((300,))) for w in i])/(len(i)+0.001)
    else:
        v = np.zeros((300,))

    sentence_vectors.append(v)

In [ ]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
# Specify number of sentences to form the summary
sn = 8

# Generate summary
for i in range(sn):
    print(ranked_sentences[i][1])